<h1 align="center">Frenet Corridor Planner Tutorial</h1>

<div align="center">
<img src="./images/img-motivation.png" alt="Motivation" width="300"/>
</div>

Motivated by the need for both effectiveness and efficiency, **path-speed decomposition-based trajectory planning** has become a standard approach in autonomous driving. While a global route can be computed offline, real-time generation of adaptive local paths remains critical for safe operation.

To address this, we present the **Frenet Corridor Planner (FCP)**—an optimization-based local path planning method designed for smooth and safe navigation around static and dynamic obstacles. In FCP, vehicles are modeled as safety-augmented bounding boxes, and pedestrians are represented using convex hulls, all within the Frenet frame. By determining the appropriate deviation side for each static obstacle, FCP constructs a drivable corridor that respects safety constraints.

A modified space-domain bicycle kinematics model is then used to optimize the path, balancing smoothness, boundary clearance, and risk avoidance from dynamic obstacles. The optimized path is subsequently passed to a speed planner to generate the final spatiotemporal trajectory.

This tutorial walks through each component of FCP and demonstrates its performance in replanning scenarios.

**Full paper**: [FCP on arXiv](https://arxiv.org/abs/2505.03695)



## Tutorial Overview
* [Example Scenario](#example-scenario)
* [Boundary Generation](#boundary-generation)
* [Sample FCP Solution](#sample-fcp-solution)
* [Dynamic Obstacle Handling](#dynamic-obstacle-handling)
* [Replanning with Perception Noise](#replanning-with-perception-noise)

## Example Scenario

We begin with a simple corridor-passing scenario involving static obstacles positioned on either side of the path.

> **Note:** All coordinates and visualizations are expressed in the Frenet frame.


In [ ]:
from utils import *
# Ego vehicle parameters
ego = [0,0,0] # x, y, yaw
ego_l, ego_w = 5, 2 # Length and width of the ego vehicle
lon_buffer = 3  # expand bounding box to make the deviated path safer
lat_buffer = 1.5 # expand bounding box to make the deviated path safer

# Obstacle parameters
obs_l, obs_w = 5, 2
stat_obss = [] # static obstacles
obs1 = [30, 0, np.deg2rad(10), obs_l, obs_w] # x, y, yaw, length, width
obs2 = [40, -1, np.deg2rad(7.5), obs_l, obs_w] # x, y, yaw, length, width
obs3 = [60, 4, np.deg2rad(-10), obs_l, obs_w] # x, y, yaw, length, width
stat_obss.extend([obs1,obs2,obs3])
dyn_obss = [] # dynamic obstacles
obss = stat_obss + dyn_obss

# Add pedestrians
N_peds = 30
peds = [] # pedestrians
random.seed(8)  # For reproducibility
for i in range(int(N_peds/2)):
    peds.append([random.randint(75,120),random.randint(-2,0)]) 
    peds.append([random.randint(75,120),random.randint(4,6)]) 

vis_scenario(ego,[],[],stat_obss,dyn_obss,[],peds=peds,length=ego_l, width=ego_w)

## Boundary Generation

Boundaries define the lower and upper limits (i.e., **corridor**) for **FCP**, which specifies the valid set of path solutions. Boundary generation involves three key steps:

1. **Bounding Box Generation for Static Obstacles**  
   For each static obstacle, a bounding box is generated. The robustness of this box depends on the perception system’s accuracy. A practical approach is to use *probabilistic bounding boxes* based on a chosen quantile to account for uncertainty.
   
   ![Bounding Box](./images/img-bounding-box.png)

2. **Classification into Lower and Upper Boundaries**  
   Each bounding box is classified as contributing to either the lower or upper boundary, based on estimated gaps relative to the ego vehicle’s position. This step determines which side of the corridor the obstacle constrains. When an obstacle is marginally positioned near the centerline, it can be treated as a risk (soft constraint) in FCP rather than a hard constraint to maintain feasibility and consistency in boundary generation.
   
   ![Classification](./images/img-bound-classification.png)

3. **Boundary Construction**  
   Based on the classification, the lower and upper boundaries are constructed. If multiple bounding boxes overlap (especially for pedestrians), a *convex hull* is computed to form a unified boundary. FCP does not require the boundaries to be smooth or linear—any time-efficient method may be used. In this tutorial, we apply a simple *extrema-based* approach that quickly identifies non-smooth boundary shapes.
   
   ![Construction](./images/img-boundary-construction.png)

> **Note:** Boundaries are generated with respect to the **ego vehicle’s center position**. Therefore, bounding boxes must include sufficient buffer space to account for the vehicle’s dimensions.


In [ ]:
# Generate boundaries (lb, ub) for the static obstacles
# The boundaries are generated based on the bounding boxes of the static obstacles.
# The bounding boxes are generated with a buffer to ensure safety.
# The bounding boxes are then classified into lower and upper boundaries based on the gap.
# TODO:: Update the function to cluser the pedestrians to a convex hull
lb,ub,bboxs = update_bounds(ego[0], stat_obss, peds=peds)

# Visualize the boundaries
vis_scenario(ego,lb,ub,stat_obss,dyn_obss,bboxs, peds)

## Sample FCP Solution

Given the previously defined lower (red) and upper (blue) boundaries, FCP solves a nonlinear optimization problem using a bicycle kinematics model with a fixed longitudinal step size.

The optimization problem is defined as follows:

$$
\begin{align}
    \min_{u}\;\; &d^\top Q_d d + u^\top Q_u u + \lambda_\text{curve} \sum_k \tan^2 u_k\ \nonumber\\ 
    &+ \lambda_\text{risk} \sum_k \left(d_k - \frac{d^\text{lb}_{t_k} + d^\text{ub}_{t_k}}{2} \right)^2 
    + \lambda_\text{dyn} \sum_{i=1}^{N_t^{\text{dyn}}} \sum_k \frac{1}{\left(\hat{d}^{(i)}_k - d_k\right)^2}, \\
    \text{subject to:} \nonumber \\
    s_{k+1} &= s_k + \Delta s, \\
    d_{k+1} &= d_k + \tan(\phi_k + u_k)\, \Delta s, \\
    \phi_{k+1} &= \phi_k + \frac{\Delta s}{\ell_r} \cdot \frac{\sin u_k}{\cos(\phi_k + u_k)}, \\
    d^\text{lb}_{t_k} - \alpha_k &\leq d_k \leq d^\text{ub}_{t_k} + \alpha_k, \\
    s_0 &= \hat{s}_t,\quad d_0 = \hat{d}_t,\quad \phi_0 = \hat{\phi}_t, \\
    u_k^\text{min} &\leq u_k \leq u_k^\text{max}.
\end{align}
$$

This formulation accounts for:

- **Smoothness and control effort** via quadratic penalties on $d$ and $u$
- **Path curvature** using a penalty on $\tan^2 u_k$
- **Obstacle clearance risk** by encouraging proximity to the centerline between the lower and upper boundaries
- **Dynamic obstacle avoidance** through inverse distance penalties

> **Note:** Please refer to the [full paper](https://arxiv.org/abs/2505.03695) for detailed derivation and implementation.


In [ ]:
# Solve FCP
sol = fcp_simple(ego, lb[:,1], ub[:,1], S=100, ds=1)

vis_scenario(ego,lb,ub,stat_obss,[],bboxs,peds=peds)
plt.plot(sol[0],sol[1],label='FCP',linewidth=65, alpha=0.5);

## Dynamic Obstacle Handling

Treating dynamic obstacles as static (i.e., imposing hard constraints) is generally unsuitable in FCP, as it can lead to infeasible solutions—especially when dynamic obstacles are nearby. Instead, FCP incorporates dynamic obstacles as **soft constraints** by adding a **risk penalty** term to the objective function.

It’s important to note that FCP operates purely in the **spatial (path) domain**—there is no explicit representation of time. However, dynamic obstacles inherently involve temporal behavior. To bridge this gap, we approximate temporal influence by **aggregating the predicted positions** of each dynamic obstacle over a fixed number of future steps ($N$) within the planning horizon.

This approach enables time-aware risk avoidance without requiring full spatiotemporal optimization.

> FCP relies on a downstream speed planner to appropriately reduce speed and prevent collisions.

In [ ]:
# Add static and dynamic obstacles
obss = [[21, -0.5, np.deg2rad(-2), 0, 5, 2], # x, y, yaw, speed, length, width
        [71, -0.5, np.deg2rad(2), 0, 5, 2], # x, y, yaw, speed, length, width
        [81, 3.5, np.deg2rad(180), 5, 5, 2]] # x, y, yaw, speed, length, width
stat_obss = [obs for obs in obss if obs[3] == 0] # static obstacles
dyn_obss = [obs for obs in obss if obs[3] != 0] # dynamic obstacles
lb,ub,bboxs = update_bounds(ego[0], stat_obss)

# Dynamic obstacles are added to the scenario with a prediction
# Here, we assume a simple linear prediction for dynamic obstacles -- this can be replaced with a more sophisticated prediction model.
# Generate predictions for dynamic obstacles over N time steps
N_pred = 10 # prediction time steps
dyn_preds = []
for obs in dyn_obss:
    preds = np.array([[obs[0] + obs[3]*k*1*np.cos(obs[2]),obs[1]+obs[3]*k*1*np.sin(obs[2]),obs[2],obs[3]] for k in range(N_pred)]) # over N time steps
    dyn_preds.append(preds)

# Solve FCP
sol = fcp_simple([0,0,0], lb[:,1], ub[:,1], S=100,dyn_preds=dyn_preds)

vis_scenario(ego,lb,ub,stat_obss,dyn_obss,bboxs)
plt.plot(sol[0],sol[1],label='FCP',linewidth=65, alpha=0.5);


From the above example, the ego vehicle tries to be distant to the expected path of the dynamic obstacle, as we should expect.

## Replanning with Perception Noise

In real-world scenarios, various sources of noise—such as perception inaccuracies and the unpredictable behavior of other traffic agents—can affect planning. This example demonstrates FCP's ability to produce smooth and consistent behavior despite **perception noise**, which is visualized as jittery boundaries.

Replanning results, including images and a video, are saved by default to `./fcp-sim/`.

> **Note:** This simulation does not include ego speed planning; the ego vehicle moves at a fixed speed of 1 m/s.


In [ ]:
import copy
from datetime import datetime
import os, random

# ====================================
# Scenario
# ====================================
ego_state = [0,0,0] # x, y, yaw
obss = [
        [20, -2, np.deg2rad(-2), 0, 5, 2], # x, y, yaw, speed, length, width
        [30, -0.5, np.deg2rad(2), 0, 5, 2], # x, y, yaw, speed, length, width
        [70, -0.5, np.deg2rad(180), 0, 5, 2], # x, y, yaw, speed, length, width
        [100, 3.5, np.deg2rad(180), 1.2, 5, 2], # x, y, yaw, speed, length, width
    ] 

# ====================================
# Control params
# ====================================
S = 50 # planning horizion [m]
ds = 1 # step size [m]
prev_path = False
perception_noise = True
smooth_out = False # Moving average smoothing
sim_N = 50 # Length of simulation
sim_dt = 1 # Simulation time step [s]
wb_buffer = 0.5 # Buffer for the which_bound check
save_figs = True # Save figures
N_pred = 10 # Prediction time steps for dynamic obstacles
occlusions = False # Consider occlusions in the scenario -- NOTE:: Setting to True will significantly increase the computation time

# ====================================
# Default params 
# ====================================
ref_lb = -2 # reference path lb
ref_ub = 5 # reference path ub
lane_lb = -2 # current lb
lane_ub = 2 # current ub
lane_min_val = -2 # hard limit for the drivable space
lane_max_val = 5 # hard limit for the drivable space

# ====================================
# Initialize
# ====================================
prevs = []
prev_sol = False
stat_obss = [obs for obs in obss if obs[3] == 0]
dyn_obss = [obs for obs in obss if obs[3] != 0]
stat_obss_raw = copy.deepcopy(stat_obss)
dyn_obss_raw = copy.deepcopy(dyn_obss)
comp_time = []
if save_figs:
    if sim_N != 0:
        now = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
        folder_name = "./fcp-sim/{}".format(now)
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

# ====================================
# Replanning
# ====================================
for k in range(sim_N): 
    if occlusions:
        stat_obss = get_visible_obss(ego_state, stat_obss)
    
    # Update bounds
    (lb,ub,bboxs,lane_lb,lane_ub) = adaptive_bounds(ego_state, stat_obss, 
                                  lane_lb=lane_lb, lane_ub=lane_ub, ref_ub=ref_ub,
                                  lane_min_val = lane_min_val, lane_max_val = lane_max_val, S=S, ds = ds,
                                  evaluate=True, wb_buffer=wb_buffer)
    
    # Get prediction on others
    # NOTE:: add points to reflect the shape of obstacles that are considered as risks
    dyn_preds = []
    for obs in dyn_obss:
        # Prediction here
        preds = np.array([[obs[0] + obs[3]*k*1*np.cos(obs[2]) + 0.5*(np.random.uniform()-0.5) * perception_noise, 
                           obs[1] + obs[3]*k*1*np.sin(obs[2]) + 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[2] + 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[3],
                           ] for k in range(N_pred)]
                        +[[obs[0]-obs[-2]/2*np.cos(obs[2])+ 0.5*(np.random.uniform()-0.5) * perception_noise,
                           obs[1]-obs[-1]/2*np.sin(obs[2]) + 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[2]                        + 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[3]],
                        [obs[0]+obs[-2]/2*np.cos(obs[2])+ 0.5*(np.random.uniform()-0.5) * perception_noise,
                         obs[1]+obs[-1]/2*np.sin(obs[2])+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                         obs[2]+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                         obs[3]]]
                        +[[obs[0]-obs[-2]*np.cos(obs[2])+ 0.5*(np.random.uniform()-0.5) * perception_noise,
                           obs[1]-obs[-1]/2*np.sin(obs[2])+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[2]+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                           obs[3]],
                        [obs[0]+obs[-2]*np.cos(obs[2])+ 0.5*(np.random.uniform()-0.5) * perception_noise,
                         obs[1]+obs[-1]/2*np.sin(obs[2])+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                         obs[2]+ 0.2*(np.random.uniform()-0.5) * perception_noise,
                         obs[3]]]
                           ) 
        dyn_preds.append(preds)

    if prev_path:
        # NOTE:: the prev_path is a list of two lists, where the first list is x and the second list is y
        # NOTE:: it is assumed that ego is at the first point of the prev_path for simplicity.
        prev_sol = [prev_path[0][1:],prev_path[1][1:]]

    # Run FCP
    sol, elapsed = fcp_simple(ego_state, lb[:,1], ub[:,1], S=S, ds=ds, dyn_preds=dyn_preds,  prev_sol = prev_sol, return_time = True); 
    comp_time.append(elapsed)
    
    # Smoothing filter
    if smooth_out and len(prevs)>0:
        prev_path = smooth_path(prevs,sol,3)
    else:
        prev_path = sol
    prevs.append(prev_path)
    if len(prevs) > 5:
        prevs.pop(0)

    # Visualize the solution
    if occlusions:
        vis_occlusions(ego_state,lb,ub,stat_obss_raw, stat_obss, bboxs, dyn_preds=dyn_preds)
    else:
        vis_scenario(ego_state,lb,ub,stat_obss,dyn_obss,bboxs, dyn_preds =dyn_preds)
    plt.plot(prev_path[0],prev_path[1],label='FCP',linewidth=65, alpha=0.5)
    plt.text(0,5.5, "Avg comp. time {} max {}".format(round(np.mean(comp_time)*1000)/1000,round(max(comp_time)*1000)/1000), fontsize=20)
    plt.legend()

    # Save the plots for visualization
    fn = "{}/{}.png".format(folder_name,k)
    with open(fn,"w") as f:
        plt.savefig(fn)
    plt.close() # clear the figure for the next iteration

    # Simulate EGO
    lookahead = 4
    vec1 = [np.cos(ego_state[2]),np.sin(ego_state[2])]
    vec2 = [prev_path[0][lookahead]-ego_state[0],prev_path[1][lookahead]-ego_state[1]]
    delta_heading = np.arccos(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
    arctan_heading = np.arctan(vec2[1]/vec2[0])
    ego_state[0] = np.interp(sim_dt,[0,ds],[prev_path[0][0],prev_path[0][1]]) 
    ego_state[1] = np.interp(sim_dt,[0,ds],[prev_path[1][0],prev_path[1][1]]) 
    ego_state[2] = arctan_heading

    # Simulate TRAFFIC
    random.seed(7)
    for obs in dyn_obss:
        # Simulate here
        obs[0] += obs[3]*sim_dt*np.cos(obs[2])
        obs[1] += obs[3]*sim_dt*np.sin(obs[2])
        obs[2] = obs[2] 
        obs[3] = obs[3] 

    if perception_noise:
        # Add noise to static obstacles
        for (obs,raw) in zip(stat_obss, stat_obss_raw):
            obs[0] = raw[0] + 0.5*(np.random.uniform()-0.5)
            obs[1] = raw[1] + 0.2*(np.random.uniform()-0.5)
            obs[2] = raw[2] + 0.2*(np.random.uniform()-0.5)

# Merge pngs to mp4 
if sim_N != 0:
    os.system("ffmpeg -framerate 5 -i '{}/%d.png' -c:v libx264 -r 30 -pix_fmt yuv420p '{}/out.mp4' -loglevel quiet".format(folder_name,folder_name))
    print("Mean comp time {} max {} min {} std {}".format(np.mean(comp_time),max(comp_time),min(comp_time),np.std(comp_time)))
    print("Video saved to {}/out.mp4".format(folder_name))